# Prompt Gym

by [Inspired Cognition](https://inspiredco.ai)

Recently, many AI-based applications are based on large language models and *prompting*, where you write a textual prompt and the model generates a response. However, app engineers and researchers are often faced with a hard decision of *which prompt to use* and *which model to use it with*.

Prompt Gym is a Jupyter notebook that demonstrates how to simply play around with prompts for text generation, testing **different models** and **different prompts** and evaluating according to **different criteria**.

<p align="center">
<img src="prompt-gym.png"  width="256" height="256">
</p>

In the example here we test two different companies' text generation models, [OpenAI's GPT-3](https://openai.com/blog/gpt-3-apps/), and [Cohere's text generation models](https://cohere.ai/generate). Evaluation of the models is done with the [Inspired Cognition Critique](https://docs.inspiredco.ai/critique/) tool for text generation evaluation. We demonstrate the case for text summarization on 10 examples from the [CNN-DailyMail dataset](https://huggingface.co/datasets/cnn_dailymail). But you can swap in whatever models, prompts, metrics, and data that you would like to try on other tasks too!

By the end of the exploration, you will have a **comprehensive report** of which prompts and models work well along a number of axes, like the actual table below that was generated from this notebook:

| Model | Prompt | UniEval (Relevance) | UniEval (Consistency) | UniEval (Coherence) | UniEval (Fluency) | BartScore (Coverage) | Length Ratio |
| --- | --- | --- | --- | --- | --- | --- | --- |
| cohere_command_xlarge | standard | 0.9144 | 0.8998 | 0.9085 | 0.9474 | -3.1686 | 1.1700 |
| openai_davinci_003 | concise | 0.9063 | 0.8328 | 0.9373 | 0.8819 | -3.0728 | 2.5300 |
| cohere_command_xlarge | complete | 0.8978 | 0.9146 | 0.9299 | 0.9371 | -3.3028 | 1.3867 |
| openai_davinci_003 | standard | 0.8974 | 0.8760 | 0.9489 | 0.9232 | -3.1167 | 2.6200 |
| cohere_command_xlarge | tldr | 0.8954 | 0.8753 | 0.9086 | 0.9557 | -3.3080 | 1.0000 |
| cohere_command_xlarge | concise | 0.8899 | 0.9006 | 0.9109 | 0.9475 | -3.3431 | 0.9333 |
| openai_gpt_3.5_turbo | standard | 0.8628 | 0.9129 | 0.9757 | 0.8902 | -3.0634 | 2.6233 |
| openai_davinci_003 | complete | 0.8568 | 0.9153 | 0.9412 | 0.9347 | -3.1098 | 2.6667 |
| openai_davinci_003 | tldr | 0.8232 | 0.9242 | 0.9490 | 0.9504 | -3.0339 | 2.5700 |
| openai_gpt_3.5_turbo | tldr | 0.8008 | 0.8856 | 0.9430 | 0.8988 | -3.1236 | 2.6000 |
| openai_gpt_3.5_turbo | complete | 0.7859 | 0.8820 | 0.9509 | 0.9045 | -3.0904 | 2.6967 |
| openai_gpt_3.5_turbo | concise | 0.7108 | 0.8712 | 0.8674 | 0.8798 | -3.0555 | 2.6033 |

Some pointers into how to do **further exploration** into the results, such as finding examples where a particular method is doing well or poorly, or where one method is outperforming the other.

If you want to discuss more, you can join the [Inspired Cognition Discord](https://discord.com/invite/vJHdpCBqWN) or get in touch through our [contact page](https://inspiredco.ai/contact/), we love talking about applications of generative AI!

## Setup

First, we import the necessary libraries and set up our API keys.

To install the requirements, run:

```bash
pip install -r requirements.txt
```

You can get the necessary API keys here:
* [OpenAI API Key](https://openai.com/blog/openai-api/)
* [Cohere API Key](https://cohere.ai/)
* [Inspired Cognition API Key](https://dashboard.inspiredco.ai)

Then, create a file called `.env` in the same directory as this notebook, and add the following lines (with ... replaced by your API keys)

```
OPENAI_API_KEY=...
COHERE_API_KEY=...
INSPIREDCO_API_KEY=...
```

Finally execute the following cell to set everything up:

In [ ]:
import os
import json
import time

import cohere
import inspiredco.critique
import openai
import tqdm

# Load environment variables from a .env file
import dotenv
dotenv.load_dotenv()

# Set up API credentials
openai.api_key = os.environ["OPENAI_API_KEY"]
cohere_api_key = os.environ["COHERE_API_KEY"]
co = cohere.Client(cohere_api_key)
inspiredco_api_key = os.environ["INSPIREDCO_API_KEY"]
critique = inspiredco.critique.Critique(inspiredco_api_key)

## Choosing Models

Next, you'll want to decide which models and configurations you'll want to use. These should follow the configuration supported by the provider. You can see more info about the generation APIs supported by each here:

* [OpenAI API Doc](https://beta.openai.com/docs/api-reference/completions/create)
* [Cohere API Doc](https://cohere.ai/docs/api/)

For demonstration purposes, we use the smaller versions of each model (`text-babbage-001` and `text-ada-001` for OpenAI, and `medium` for Cohere), but you can change these to the larger versions (`text-davinci-003` for OpenAI and `xlarge` for Cohere) if you want to try them out.

In [ ]:
# Specify which models you want to use
models = {
    "cohere_command_xlarge": {
        "provider": "cohere",
        "config": {
            "model": "command-xlarge-nightly",
            "temperature": 0.3,
            "max_tokens": 100,
            "top_p": 1,
        }
    },
    "openai_davinci_003": {
        "provider": "openai",
        "config": {
            "model": "text-davinci-003",
            "temperature": 0.3,
            "max_tokens": 100,
            "top_p": 1,
        }
    },
    "openai_gpt_3.5_turbo": {
        "provider": "openai_chat",
        "config": {
            "model": "gpt-3.5-turbo",
            "temperature": 0.3,
            "max_tokens": 100,
            "top_p": 1,
        }
    },
}

## Choosing Prompts

Next, you will want to choose prompts. For the prompts, the input text will be input into the placeholder `[X]`.

The examples below are for text summarization, but you can change the prompts to do other tasks as well.

In [ ]:
# Specify the prompts you want to use
prompts = {
    "standard": "Summarize the following text:\n[X]\n\nSummary:",
    "tldr": "[X]\nTL;DR:",
    "concise": "Write a short and concise summary of the following text:\n[X]\n\nSummary:",
    "complete": "Write a complete summary of the following text:\n[X]\n\nSummary:",
}

## Choosing Evaluation Metrics

Finally, you'll want to decide which metrics you use to evaluate the quality of the generated text. Critique supports a wide variety of metrics, so you'll want to pick appropriate ones for your task. You can read more about this on the following pages:
* [Critique Evaluation Criteria](https://docs.inspiredco.ai/critique/criteria.html)
* [Critique Metrics](https://docs.inspiredco.ai/critique/metrics.html)

In [ ]:
metrics = {
    "UniEval (Relevance)": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "relevance"},
    },
    "UniEval (Consistency)": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "consistency"},
    },
    "UniEval (Coherence)": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "coherence"},
    },
    "UniEval (Fluency)": {
        "metric": "uni_eval",
        "config": {"task": "summarization", "evaluation_aspect": "fluency"},
    },
    "BartScore (Coverage)": {
        "metric": "bart_score",
        "config": {"model": "facebook/bart-large-cnn", "variety": "reference_target_bidirectional"},
    },
    "Length Ratio": {
        "metric": "length_ratio",
        "config": {},
    },
}

## Set up Data

Now we'll load our data! Put the inputs in a jsonl file, where the input is in the `source` field and a gold-standard output is in the `reference` field.
The source and target examples in this repo are 10 documents and summaries from the CNN-DailyMail dataset, but you can swap in whatever data you want. You'll probably want to use more examples for robust results in practice, but we use a small number here for demonstration purposes.

In [ ]:
with open("input_data.jsonl", "r") as f:
    input_data = [json.loads(line) for line in f.readlines()]

## Generate Output Text

Iterate through the models and prompts and generate the output text. This may take a while to hit the APIs many times. This will also write out the generated data to `output_data/targets.json` so you can re-run the evaluation step later without having to generate outputs again.

In [ ]:
targets = {}
for model_name, model_info in models.items():
    config, provider = model_info["config"], model_info["provider"]
    targets[model_name] = {}
    for prompt_name, prompt_template in prompts.items():
        my_data = []
        for input in tqdm.tqdm(input_data, desc=f"Generating {model_name=} {prompt_name=}"):
            source = input["source"]
            prompt = prompt_template.replace("[X]", source)
            if provider == "openai":
                response = openai.Completion.create(
                    engine=config["model"],
                    prompt=prompt,
                    temperature=config["temperature"],
                    max_tokens=config["max_tokens"],
                    top_p=config["top_p"],
                )
                my_data.append(response["choices"][0]["text"])
            elif provider == "openai_chat":
                response = openai.ChatCompletion.create(
                    model=config["model"],
                    messages=[
                        {"role": "user", "content": prompt},
                    ],
                    temperature=config["temperature"],
                    max_tokens=config["max_tokens"],
                    top_p=config["top_p"],
                )
                my_data.append(response["choices"][0]["message"]["content"])
            elif provider == "cohere":
                try:
                    
                    response = co.generate(  
                        model=config["model"],  
                        prompt=prompt,
                        temperature=config["temperature"],  
                        max_tokens=config["max_tokens"],
                        p=config["top_p"], 
                    )
                    my_data.append(response.generations[0].text)
                except:
                    # Cohere API sometimes rejects queries, if so output a blank line
                    print(f"Warning! Cohere API rejected query for {prompt=}")
                    my_data.append("")
                time.sleep(10)  # Sleep to avoid rate limiting on developer API
            else:
                raise ValueError("Unknown provider, but you can add your own!")
        targets[model_name][prompt_name] = my_data
if not os.path.exists("output_data"):
    os.makedirs("output_data")
with open("output_data/targets.json", "w") as f:
    json.dump(targets, f, indent=2)

## Evaluate Output Text

Now we'll evaluate the output text from a number of different perspectives. We'll also save the evaluation results for later use.

In [ ]:
critique_data = {}
# Dispatch jobs
for model_name, model_data in targets.items():
    critique_data[model_name] = {}
    for prompt_name, target_data in model_data.items():
        critique_data[model_name][prompt_name] = {}
        print(f"Submitting evaluation jobs for {model_name=} {prompt_name=}")
        for metric_name, metric_info in metrics.items():
            metric, config = metric_info["metric"], metric_info["config"]
            dataset = [
                {"source": input["source"], "target": target, "references": [input["reference"]]}
                for input, target in zip(input_data, target_data)
            ]
            critique_data[model_name][prompt_name][metric_name] = critique.submit_task(
                metric=metric,
                config=config,
                dataset=dataset,
            )

# Collect results
for model_name, model_data in critique_data.items():
    for prompt_name, prompt_data in model_data.items():
        print(f"Retrieving evaluation results for {model_name=} {prompt_name=}")
        for metric_name, task_id in prompt_data.items():
            prompt_data[metric_name] = critique.wait_for_result(task_id)
with open("output_data/evaluations.json", "w") as f:
    json.dump(critique_data, f, indent=2)

## Report Evaluation Table

Finally, we'll generate a table of the evaluation results, and save it to `output_data/evaluation_table.md`, as well as displaying it here.

In [ ]:
# Sort by the first metric by default, but you can change this if you want
sort_metric = list(metrics.keys())[0]
with open("output_data/evaluation_table.md", "w") as f:
    def print_both(text):
        print(text)
        print(text, file=f)
    metric_list = list(metrics.keys())
    all_rows = []
    for model_name, model_data in critique_data.items():
        for prompt_name, prompt_data in model_data.items():
            row = [model_name, prompt_name]
            for metric_name, metric_data in prompt_data.items():
                row.append(metric_data['overall']['value'])
            all_rows.append(row)
    headers = ["Model", "Prompt"] + metric_list
    sort_column = headers.index(sort_metric)
    all_rows.sort(key=lambda row: row[sort_column], reverse=True)
    print_both("| " + " | ".join(headers) + " |")
    print_both("| " + " | ".join("---" for _ in headers) + " |")
    for row in all_rows:
        row = row[:2] + [f"{x:.4f}" for x in row[2:]]
        print_both("| " + " | ".join(row) + " |")

## Further Exploration

You can also further explore the results on an example-by-example basis for a particular model, prompt, and metric. First, let's specify the model, prompt, and metric we want to explore.

In [ ]:
model = "openai_gpt_3.5_turbo"
prompt = "standard"
metric = "UniEval (Relevance)"

### Finding High/low-performing Examples

Here's an example of how you can sort the outputs and find particularly high-performing or low-performing examples to do error analysis. Here we're only outputting the system outputs and references because the summarization sources are long, but you could also print out the sources if you wanted to.

In [ ]:
metric_data = [x["value"] for x in critique_data[model][prompt][metric]["examples"]]
target_data = targets[model][prompt]
graded_data = list(zip(input_data, target_data, metric_data))
graded_data.sort(key=lambda x: x[2])

def print_graded(input, target, metric_value):
    print(f"=== {metric_value:0.4f} ===")
    print("*** Reference ***")
    print(input["reference"])
    print()
    print("*** Target ***")
    print(target)
    print()

print(f"----- Worst {metric} Examples for '{model} {prompt}' -----")
for input, target, metric_value in graded_data[:3]:
    print_graded(input, target, metric_value)

print(f"----- Best {metric} Examples for '{model} {prompt}' -----")
for input, target, metric_value in graded_data[-3:]:
    print_graded(input, target, metric_value)


### Finding Examples Where One Method Outperforms Another

When working with prompting, we'll often want to know which prompts are better than others and in which ways. First, let's specify two methods we're interested in:

In [ ]:
model1 = "openai_gpt_3.5_turbo"
prompt1 = "concise"
model2 = "openai_gpt_3.5_turbo"
prompt2 = "tldr"
metric = "UniEval (Relevance)"

Then we can find examples where one method outperforms the other:

In [ ]:
metric_data1 = [x["value"] for x in critique_data[model1][prompt1][metric]["examples"]]
target_data1 = targets[model1][prompt1]
metric_data2 = [x["value"] for x in critique_data[model2][prompt2][metric]["examples"]]
target_data2 = targets[model2][prompt2]
graded_data = list(zip(input_data, target_data1, target_data2, metric_data1, metric_data2))
graded_data.sort(key=lambda x: x[3]-x[4], reverse=True)

def print_graded(input, target1, target2, metric_value1, metric_value2):
    print(f"=== {metric_value1:0.4f} vs. {metric_value2:0.4f} ===")
    print("*** Reference ***")
    print(input["reference"])
    print()
    print("*** Target 1 ***")
    print(target1)
    print()
    print("*** Target 2 ***")
    print(target2)
    print()

print(f"----- Examples where '{model1} {prompt1}' outperforms '{model2} {prompt2}' on {metric} -----")
for input, target1, target2, metric_value1, metric_value2 in graded_data[:3]:
    print_graded(input, target1, target2, metric_value1, metric_value2)

print(f"----- Examples where '{model2} {prompt2}' outperforms '{model1} {prompt1}' on {metric} -----")
for input, target1, target2, metric_value1, metric_value2 in graded_data[-3:]:
    print_graded(input, target1, target2, metric_value1, metric_value2)

## Final Words

That's it for today! Hope you had a good prompting workout, and if you have any comments, questions, or suggestions, drop us a line in [discord](https://discord.com/invite/vJHdpCBqWN) or through our [contact page](https://inspiredco.ai/contact/).